In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [7]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
today = date.today()

In [8]:
def read_inv_data(COMPANY_IDENTIFIER,license_numbers):
    
    company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
    )
    company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
    return company_inventory_packages_query,company_inventory_packages_dataframe,


In [9]:
inv_query_99ht, df_inventory_99ht = read_inv_data(['99HT'],['C10-0000279-LIC'])

In [10]:
print(inv_query_99ht)


		select
			metrc_packages.license_number,
			metrc_packages.package_id,
			metrc_packages.package_label,
			metrc_packages.type,
			metrc_packages.packaged_date,
			metrc_packages.last_modified_at,
			metrc_packages.package_type,
			metrc_packages.product_name,
			metrc_packages.product_category_name,
			metrc_packages.quantity,
			metrc_packages.unit_of_measure,
			metrc_packages.package_payload.itemid as item_id,
			metrc_packages.package_payload.itemproductcategorytype as item_product_category_type,
			metrc_packages.package_payload.productionbatchnumber as production_batch_number,
			metrc_packages.package_payload.sourceproductionbatchnumbers as source_production_batch_numbers,
			metrc_packages.package_payload.sourceharvestnames as source_harvest_names,
			metrc_packages.package_payload.istestingsample as is_testing_sample,
			metrc_packages.package_payload.istradesample as is_trade_sample,
			metrc_packages.package_payload.isonhold as is_on_hold,
			metrc_packages.package_paylo

In [11]:
df_inventory_99ht.head()

,license_number,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date
0,C10-0000279-LIC,24482415,1A40603000072DE000236294,active,2022-04-21,2022-04-22 18:38:28+00:00,Product,RA' Whole Flower Multi-Use THC Capsules (50MG ...,Capsule (weight - each),108.0,Each,4403818,Concentrate,,102,"Moon Drops 10/12 Second Cut, Moon Drops 10/31 ...",False,False,False,None,None
1,C10-0000279-LIC,24482313,1A40603000072DE000236295,active,2022-04-21,2022-04-22 18:38:28+00:00,Product,RA' Whole Flower Multi-Use THC Capsules (25MG ...,Capsule (weight - each),96.0,Each,4475001,Concentrate,,105,"Lemon Jack 10/13 First Cut, Lemon Jack 10/19 H...",False,False,False,None,None
2,C10-0000279-LIC,24422074,1A4060300009FD9000009420,active,2022-04-19,2022-04-19 19:10:20+00:00,Product,Vanilla Waferz - 1g - Mylar,Flower (packaged gram - each),13.0,Each,3937096,Buds,,,"Mclane 5,6 - Ice Cream Cake 8 - 9/13/2021",False,False,False,None,None
3,C10-0000279-LIC,24422072,1A4060300009FD9000009418,active,2022-04-19,2022-04-19 19:10:20+00:00,Product,Dripp Live Resin Disposable 0.5 gram (Snow Man),Other Concentrate (weight - each),13.0,Each,3829970,Concentrate,,DB/SM/DrT102921,"2020-10-18-Dry Pig Pond Garage-H, CUN-AS-Snow,...",False,False,False,None,None
4,C10-0000279-LIC,24422073,1A4060300009FD9000009419,active,2022-04-19,2022-04-19 19:10:20+00:00,Product,Banana Waferz - 1g - Mylar,Flower (packaged gram - each),12.0,Each,3937094,Buds,,,"555 1,2 - PopRox - 9/13/2021, FedEx 1-4 - PopR...",False,False,False,None,None


# Inventory Valuation

In [12]:
def read_df_in(COMPANY_IDENTIFIER,license_numbers):
    company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [13]:
def read_df_sales(COMPANY_IDENTIFIER,license_numbers):
    query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
    )
    company_sales_receipts_with_transactions_dataframe = pd.read_sql_query(query, engine)
    deduped_sales_receipts_with_transactions_dataframe = prepare_data.dedupe_sales_transactions(company_sales_receipts_with_transactions_dataframe)
    deduped_sales_receipts_with_transactions_dataframe['sales_datetime'] = pd.to_datetime(deduped_sales_receipts_with_transactions_dataframe['sales_datetime'])
    deduped_sales_receipts_with_transactions_dataframe['sales_month'] = deduped_sales_receipts_with_transactions_dataframe['sales_datetime'].dt.strftime('%Y-%m')
    return deduped_sales_receipts_with_transactions_dataframe



In [14]:
df_in_99ht = read_df_in(['99HT'],['C10-0000279-LIC'])
df_sales_99ht = read_df_sales(['99HT'],['C10-0000279-LIC'])

In [15]:
def calculate_inventory_valuation(incoming_transfer_df,inventory_df,license_list):
    # legal name
    legal_name = incoming_transfer_df[incoming_transfer_df['license_number'].isin(license_list)]['recipient_facility_name'].values[0]
    # process df_in and df_sales
    incoming_transfer_df['per_unit_incoming'] = incoming_transfer_df['shipper_wholesale_price'] / incoming_transfer_df['shipped_quantity']
    incoming_transfer_df_price = incoming_transfer_df[incoming_transfer_df['shipper_wholesale_price'].notnull()]
    # by package id
    average_incoming_package_id = incoming_transfer_df_price.groupby(['package_id'])['per_unit_incoming'].mean()
    df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
    df_avg_incoming_price = df_avg_incoming_price.reset_index()
    # by product
    average_incoming_product = incoming_transfer_df_price.groupby(['product_name'])['per_unit_incoming'].mean()
    df_avg_product = pd.Series(average_incoming_product).to_frame()
    df_avg_product = df_avg_product.reset_index()
    df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)
    #calculate inventory
    df_inventory_incoming = pd.merge(inventory_df, df_avg_incoming_price, left_on=['package_id'],right_on = ['package_id'], how='left')
    #left_on=['tx_product_name','tx_unit_of_measure'], right_on=['product_name','shipped_unit_of_measure'], how='left'
    df_inventory_incoming.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_null = df_inventory_incoming[df_inventory_incoming['per_unit_incoming'].isnull()]
    df_inv_product = pd.merge(df_inv_null, df_avg_product, left_on=['product_name'],right_on = ['product_name'], how='left')
    df_inv_product.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_product_price = df_inv_product[df_inv_product['per_unit_product'].notnull()]
    df_inv_product_price['total_price'] = df_inv_product_price['quantity'] * df_inv_product_price['per_unit_product']
    
    inventory_product_value = df_inv_product_price['total_price'].sum()
    df_inventory_incoming['total_price'] = df_inventory_incoming['quantity'] * df_inventory_incoming['per_unit_incoming']
    inventory_value = df_inventory_incoming['total_price'].sum()
    total_inv_value = inventory_product_value + inventory_value
    total_inv_value_after_tax = (inventory_product_value + inventory_value) * 1.27
    inv_count_product = df_inv_product_price['per_unit_product'].count()
    inv_count_incoming = df_inventory_incoming['per_unit_incoming'].count()
    inv_count_total = df_inventory_incoming['quantity'].count()
    inv_total_incoming = inv_count_product + inv_count_incoming
    inventory_coverage = inv_total_incoming / inv_count_total
    # prepare data
    data = [[today], 
            [total_inv_value], 
            [total_inv_value_after_tax],
            [inv_total_incoming], 
            [inv_count_total], 
            [inventory_coverage],
            [license_list],
            [legal_name]]
    df_inventory_license = pd.DataFrame(data).T
    df_inventory_license.columns = ['date','value','value_after_tax','total_incoming','total','coverage','license','legal_name']
    return df_inventory_license
    

In [16]:

df_inventory_license_99ht = calculate_inventory_valuation(df_in_99ht,df_inventory_99ht,['C10-0000279-LIC'])

In [17]:
df_inventory_license_99ht

,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-04-24,126427,160562,738,797,0.925972,[C10-0000279-LIC],99 HIGHTIDE COLLECTIVE


In [18]:
def calculate_msrp_based_inventory_valuation(incoming_transfer_df,sales_df,inventory_df,license_list):
    # legal name
    legal_name = incoming_transfer_df[incoming_transfer_df['license_number'].isin(license_list)]['recipient_facility_name'].values[0]
    # process df_in and df_sales
    sales_df['per_unit'] = sales_df['tx_total_price'] / sales_df['tx_quantity_sold']
    sales_df['year_month'] = sales_df['sales_datetime'].dt.strftime("%Y-%m")    
    # per unit msrp by package id
    df_msrp = sales_df[sales_df['tx_total_price'].notnull()]
    average_msrp_package_id = df_msrp.groupby('tx_package_id')['per_unit'].mean()
    df_avg_msrp_package_id = pd.Series(average_msrp_package_id).to_frame()
    df_avg_msrp_package_id = df_avg_msrp_package_id.reset_index()
    # per unit msrp by product name
    average_msrp_product = df_msrp.groupby('tx_product_name')['per_unit'].mean()
    df_avg_msrp_product = pd.Series(average_msrp_product).to_frame()
    df_avg_msrp_product = df_avg_msrp_product.reset_index()
    df_avg_msrp_product.rename(columns={'per_unit':'per_unit_product'}, inplace=True)
    
    #calculate inventory
    #merge with per unit msrp by package id
    df_inventory_package_id = pd.merge(inventory_df, df_avg_msrp_package_id, left_on=['package_id'],right_on = ['tx_package_id'], how='left')
    df_inventory_package_id.replace([numpy.inf], numpy.nan, inplace=True)
    # merge with per unit msrp by product
    df_inv_null = df_inventory_package_id[df_inventory_package_id['per_unit'].isnull()]
    df_inv_product = pd.merge(df_inv_null, df_avg_msrp_product, left_on=['product_name'],right_on = ['tx_product_name'], how='left')
    df_inv_product.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_product_price = df_inv_product[df_inv_product['per_unit_product'].notnull()]
    df_inv_product_price['total_price'] = df_inv_product_price['quantity'] * df_inv_product_price['per_unit_product']
    inventory_product_value = df_inv_product_price['total_price'].sum()
    
    df_inventory_package_id['total_price'] = df_inventory_package_id['quantity'] * df_inventory_package_id['per_unit']
    inventory_package_id_value = df_inventory_package_id['total_price'].sum()
    total_inv_value = inventory_product_value + inventory_package_id_value
    inv_count_product = df_inv_product_price['per_unit_product'].count()
    inv_count_package_id = df_inventory_package_id['per_unit'].count()
    inv_count_total = df_inventory_package_id['quantity'].count()
    inv_total_mapped = inv_count_product + inv_count_package_id
    inventory_coverage = inv_total_mapped / inv_count_total
    # prepare data
    data = [[today], 
            [total_inv_value], 
            [inv_total_mapped], 
            [inv_count_total], 
            [inventory_coverage],
            [license_list],
            [legal_name]
            ]
    df_inventory_license = pd.DataFrame(data).T
    df_inventory_license.columns = ['date','value','total_incoming','total','coverage','license','legal_name']
    return df_inventory_license
    

In [19]:

df_inventory_license_99ht_msrp = calculate_msrp_based_inventory_valuation(df_in_99ht,df_sales_99ht,df_inventory_99ht,['C10-0000279-LIC'])

In [20]:
df_inventory_license_99ht_msrp

,date,value,total_incoming,total,coverage,license,legal_name
0,2022-04-24,414734,668,797,0.838143,[C10-0000279-LIC],99 HIGHTIDE COLLECTIVE


In [21]:
#you can also query like this
pd.read_sql_query("""SELECT
  sum(
    metrc_packages.quantity * metrc_transfer_packages.shipper_wholesale_price / metrc_transfer_packages.shipped_quantity * 1.27
  ) AS inventory_valuation
FROM
  metrc_packages AS metrc_packages
  INNER JOIN companies AS companies ON metrc_packages.company_id = companies.id
  LEFT JOIN metrc_transfer_packages AS metrc_transfer_packages ON metrc_packages.package_id = metrc_transfer_packages.package_id
WHERE
  TRUE
  AND LOWER(companies.name) NOT LIKE '%demo%'
  AND companies.identifier = 'EMA'
  AND metrc_packages.type = 'active'
  AND metrc_transfer_packages.shipped_quantity > 0""",engine)

,inventory_valuation
0,290010.860962
